
# Neural Network Model Development

This notebook demonstrates the development of a custom neural network using TensorFlow and Keras, focusing on good coding practices and clear documentation.

### Library Imports
All necessary libraries are imported here for better organization.


In [1]:
!pip install keras

In [2]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Synthetic Sensor data

-------------------------

In [3]:
df = pd.read_csv('/content/synthetic_sensor_dataset.csv')
df.head(10)

,timestamp,linear_accel_x,linear_accel_y,linear_accel_z,angular_vel_x,angular_vel_y,angular_vel_z,joint_0_position,joint_0_velocity,joint_0_effort,...,joint_2_effort,joint_3_position,joint_3_velocity,joint_3_effort,joint_4_position,joint_4_velocity,joint_4_effort,joint_5_position,joint_5_velocity,joint_5_effort
0,2024-01-30 18:22:36,-4.942083,-4.029851,7.054618,-0.028914,-2.684058,1.755596,157.083505,4.958667,-40.461184,...,15.568628,83.380417,79.888414,44.503294,-57.588535,-65.635121,-31.786955,107.896263,-8.441808,13.705776
1,2024-01-30 18:22:37,2.443458,9.137248,0.083475,3.226335,4.616319,3.350469,152.110388,66.929192,-3.081446,...,48.597736,-170.881204,-24.946624,20.167479,-101.185134,64.669012,-20.904691,-52.832063,-10.042246,21.573037
2,2024-01-30 18:22:38,5.504044,3.926436,-0.969042,1.744536,-2.363244,1.069115,-28.341868,-52.918594,31.498440,...,45.117153,11.205067,-47.258649,-45.885168,174.369793,34.140785,18.909646,-108.721794,-79.840394,23.754550
3,2024-01-30 18:22:39,4.333194,9.088666,3.597800,-4.291289,-3.904077,-3.175750,-121.944185,-44.295597,-35.052974,...,-13.587203,158.528814,8.835446,-22.957318,179.217855,-55.310303,-33.700546,13.630050,84.226360,9.310007
4,2024-01-30 18:22:40,-1.850610,7.266473,3.652301,-1.287008,-3.273892,3.141748,32.815826,-60.515687,-44.006690,...,28.881733,177.486516,-43.350363,27.256251,25.695819,-89.400555,30.656158,-172.766939,-86.426640,43.596588
5,2024-01-30 18:22:41,-1.005813,0.216967,-9.916787,-4.835599,2.732140,3.172858,69.541937,-72.852287,6.988084,...,-28.005350,135.046262,52.355893,-33.805249,-59.445957,48.436550,28.568093,-7.471051,31.421933,-36.772546
6,2024-01-30 18:22:42,2.110933,-2.404513,-3.218972,-1.845220,-0.275860,-2.315603,-119.078730,-63.206131,-18.548739,...,26.251597,23.585870,-39.175099,-13.675407,7.234326,49.163569,-5.790125,97.320282,5.129106,-40.955385
7,2024-01-30 18:22:43,-6.983143,0.031118,2.979066,3.032103,-4.596503,1.101446,-141.939189,-40.597379,-30.409487,...,4.864105,-11.968429,-3.580947,21.646637,25.828868,40.780879,-15.685156,148.834833,63.350359,37.334450
8,2024-01-30 18:22:44,-2.652212,-1.202847,-2.161007,3.334166,-4.706792,-3.594128,-35.109996,22.768431,32.706263,...,-29.104932,-138.613910,-42.349988,24.044450,42.355410,6.523652,17.937138,-25.645948,-81.319329,16.889619
9,2024-01-30 18:22:45,-4.366013,8.434773,-6.989508,-0.682954,1.792306,-4.144608,-133.158660,-26.257355,-21.228601,...,49.646530,69.272178,-51.514623,-25.451984,101.935525,25.960054,12.555234,-11.809478,-17.035011,-4.130428



### Global Variables
Defining any constants and global variables used throughout the notebook.


In [ ]:

# Adjust these parameters as needed for your model
seq_length = 128
d_model = 512
num_classes = 10



## Custom Layer Definitions

Here we define custom layers with appropriate documentation and naming conventions.



### BoolformerLayer

This custom TensorFlow layer performs a logical AND operation on its input and then processes it through a dense layer with ReLU activation.


In [ ]:
class BoolformerLayer(layers.Layer):
    def __init__(self, threshold=0.5, **kwargs):
        super(BoolformerLayer, self).__init__(**kwargs)
        self.threshold = threshold

    def build(self, input_shape):
        self.dense_layer = layers.Dense(input_shape[-1], activation='relu')

    def call(self, inputs):
        boolean_inputs = tf.greater(inputs, self.threshold)  # Convert to boolean based on threshold
        logic_and = tf.math.logical_and(boolean_inputs, boolean_inputs)
        return self.dense_layer(tf.cast(logic_and, tf.float32))  # Convert back to float



### QLearningLayer

This layer is designed for reinforcement learning tasks, using a Q-learning algorithm to learn the quality of actions.


In [ ]:
class QLearningLayer(layers.Layer):
    def __init__(self, action_space_size, learning_rate=0.01, gamma=0.95, **kwargs):
        super(QLearningLayer, self).__init__(**kwargs)
        self.action_space_size = action_space_size
        self.learning_rate = learning_rate
        self.gamma = gamma

    def build(self, input_shape):
        # A dense layer to process state and output Q-values for each action
        self.dense = layers.Dense(self.action_space_size, activation=None)

    def call(self, state, action=None, reward=None, next_state=None):
        q_values = self.dense(state)

        if action is not None and reward is not None and next_state is not None:
            # Get the predicted Q-values for the next state
            future_q_values = self.dense(next_state)
            max_future_q = tf.reduce_max(future_q_values, axis=1)

            # Compute the updated Q-value for the chosen action
            q_update = reward + self.gamma * max_future_q
            q_values_with_update = tf.tensor_scatter_nd_update(
                q_values, tf.expand_dims(action, axis=-1), q_update)

            # Update the Q-values
            self.dense.set_weights([q_values_with_update])

        return q_values


## Helper Functions

Defining helper functions such as positional encoding and transformer encoder with detailed comments for better understanding.



### Positional Encoding Function

Positional encoding adds information about the position of elements in the input sequence, crucial for models like transformers.


In [ ]:
def positional_encoding(seq_length, d_model):
    position = tf.range(seq_length, dtype=tf.float32)[:, tf.newaxis]
    div_term = tf.exp(tf.range(0, d_model, 2, dtype=tf.float32) * -(tf.math.log(10000.0) / d_model))

    # Creating sine and cosine functions separately and then concatenating them
    sine_terms = tf.sin(position * div_term)
    cosine_terms = tf.cos(position * div_term)

    # Interleaving sine and cosine terms
    pos_encoding = tf.reshape(tf.concat([sine_terms, cosine_terms], axis=-1), [1, seq_length, d_model])

    return pos_encoding


### Transformer Encoder Function

The transformer encoder function applies transformations to the input data using layer normalization and multi-head attention, followed by a series of dense layers.


In [ ]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)
    return x + res



## Model Building and Compilation

Here we build and compile the neural network model, ensuring clarity and efficiency in the code.



### Neural Network Model Creation Function

This function constructs the neural network using the previously defined custom layers and functions. It integrates the transformer encoder with the custom `BoolformerLayer` and `QLearningLayer`.


In [ ]:
def create_neural_network_model():
    input_layer = keras.Input(shape=(seq_length, d_model))

    # Generate positional encoding and add it to the input
    pos_encoding = positional_encoding(seq_length, d_model)  # Ensure this returns a tensor
    pos_encoded = input_layer + pos_encoding

    # Transformer encoder
    transformer_output = transformer_encoder(inputs=pos_encoded, head_size=32, num_heads=2, ff_dim=64)

    # Custom layers (assuming these are correctly defined elsewhere)
    x_bool = BoolformerLayer()(transformer_output)
    rl_layer = QLearningLayer(action_space_size=num_classes)(x_bool)

    # Output layers
    output_layer = layers.Dense(num_classes, activation='softmax', name='Output')(rl_layer)
    reward_layer = layers.Dense(1, name='Reward')(rl_layer)

    # Constructing the model
    model = keras.Model(inputs=input_layer, outputs=[output_layer, reward_layer])

    # Compiling the model
    opt = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss={'Output': 'categorical_crossentropy', 'Reward': 'mean_squared_error'},
                  metrics={'Output': 'accuracy'})

    return model

# Creating the model
model = create_neural_network_model()

# Displaying the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 128, 512)]           0         []                            
                                                                                                  
 tf.__operators__.add_7 (TF  (None, 128, 512)             0         ['input_5[0][0]']             
 OpLambda)                                                                                        
                                                                                                  
 layer_normalization_4 (Lay  (None, 128, 512)             1024      ['tf.__operators__.add_7[0][0]
 erNormalization)                                                   ']                            
                                                                                              


## Hyperparameter Settings

This section defines the hyperparameters for the model. Adjust these parameters to fine-tune the model's training process.


In [ ]:

learning_rate = 0.001
batch_size = 32
epochs = 20
# Define additional hyperparameters here


## Model Training

In this section, we train the neural network model using the specified hyperparameters. The `model.fit()` function will be used to train the model with the training data. The validation data will be used to monitor the model's performance on unseen data.

In [ ]:
# Data Preparation
# Auto-detecting output and reward columns based on model architecture
output_label_column = df.columns[-2]  # Change this as per DataFrame Structure
reward_label_column = df.columns[-1]  # Change this as per DataFrame Structure
input_columns = df.drop([output_label_column, reward_label_column], axis=1)

# Standardize the input features
scaler = StandardScaler()
X = scaler.fit_transform(input_columns)

# Preparing output and reward labels
Y_output = df[output_label_column].values  # Assuming categorical labels
Y_reward = df[reward_label_column].values  # Assuming continuous values

# Splitting the dataset
X_train, X_val, Y_output_train, Y_output_val, Y_reward_train, Y_reward_val = train_test_split(
    X, Y_output, Y_reward, test_size=0.2, random_state=42
)

# Formatting data for model training
train_data = (X_train, {'Output': Y_output_train, 'Reward': Y_reward_train})
val_data = (X_val, {'Output': Y_output_val, 'Reward': Y_reward_val})

# Model Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='min'),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    train_data,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=val_data,
    verbose=1,
    callbacks=callbacks
)

# Model Saving
model.save('SephsRL.h5')

## Model Evaluation

After training the model, it's important to evaluate its performance on a test dataset to understand its efficacy. The following code will use the `model.evaluate()` function to assess the model's accuracy and loss on the test data.

In [ ]:
# Evaluate the model on the test data
# test_data = ...

evaluation_metrics = model.evaluate(test_data)
print(f"Test Loss: {evaluation_metrics[0]}, Test Accuracy: {evaluation_metrics[1]}")


## Visualizing Model Performance

Functions for plotting and analyzing the model's performance during training.


In [ ]:

import matplotlib.pyplot as plt

# Function to plot training history for both 'Output' and 'Reward' outputs, tailored to their characteristics
def plot_custom_output_history(history):
    num_plots = 2 + ('accuracy' in history.history)
    fig, axes = plt.subplots(1, num_plots, figsize=(6 * num_plots, 5))

    plot_index = 0

    # Plotting accuracy for 'Output', if it's available
    if 'accuracy' in history.history:
        axes[plot_index].plot(history.history['accuracy'], label='Training Accuracy - Output')
        axes[plot_index].plot(history.history['val_accuracy'], label='Validation Accuracy - Output')
        axes[plot_index].set_title('Accuracy for Output')
        axes[plot_index].set_xlabel('Epochs')
        axes[plot_index].set_ylabel('Accuracy')
        axes[plot_index].legend()
        plot_index += 1

    # Plotting loss for 'Output'
    axes[plot_index].plot(history.history['Output_loss'], label='Training Loss - Output')
    axes[plot_index].plot(history.history['val_Output_loss'], label='Validation Loss - Output')
    axes[plot_index].set_title('Loss for Output')
    axes[plot_index].set_xlabel('Epochs')
    axes[plot_index].set_ylabel('Loss')
    axes[plot_index].legend()
    plot_index += 1

    # Plotting loss for 'Reward'
    axes[plot_index].plot(history.history['Reward_loss'], label='Training Loss - Reward')
    axes[plot_index].plot(history.history['val_Reward_loss'], label='Validation Loss - Reward')
    axes[plot_index].set_title('Loss for Reward')
    axes[plot_index].set_xlabel('Epochs')
    axes[plot_index].set_ylabel('Loss')
    axes[plot_index].legend()

    plt.tight_layout()
    plt.show()

# Call the function with the training history
plot_custom_output_history(history)



## Conclusion

This notebook provided a detailed walkthrough for developing, training, and evaluating a neural network model with custom layers and advanced techniques, ensuring good coding practices and clear documentation throughout.
